In [110]:
import glob
import pandas as pd
import os 

In [159]:
game_sim_files = glob.glob('../output_results/game_sim/game_level/*/*/conducted*')

In [160]:
all_dfs = []
for f in game_sim_files:
    f_path, fname = os.path.split(f)
    f_path, trial_name = os.path.split(f_path)
    game_level = os.path.basename(f_path)

    df = pd.read_json(f, lines=True)
    df['trial_name'] = trial_name
    df['game_level'] = game_level
    all_dfs.append(df)

In [161]:
full_df = pd.concat(all_dfs)

In [162]:
print(full_df['info_items'].iloc[0])

Here are the key pieces of information extracted from the interview transcript:

Information item #1: The U.S. intelligence report presented to President-elect Trump states that Russia's president, Vladimir Putin, personally ordered a campaign to hurt Hillary Clinton during the U.S. presidential campaign and to try to help Donald Trump win the election.

Information item #2: The Russian government's response to the report is that it's "more of the same, nothing new" and that the report provided no evidence, with the goal of undermining the legitimacy of Trump's election.

Information item #3: Putin has previously responded to these charges, mocking the idea that Russia could have thrown a U.S. election and suggesting that the Democrats are sore losers.

Information item #4: The Russian government is not concerned about the allegations, and Putin may even see them as a positive development, as they make Russia seem more powerful and influential on the world stage.

Information item #5: 

In [184]:
full_df['total_info_items_extracted'].value_counts()

total_info_items_extracted
10    607
9     297
8     277
12    191
11    174
7      84
6      34
3      19
4      16
5      15
2       5
0       4
1       2
Name: count, dtype: int64

In [189]:
full_df.pipe(lambda df: df['total_info_items_extracted'] / df['total_info_item_count'])

0    0.500000
1    0.000000
2    0.800000
3    0.428571
4    0.000000
       ...   
0    1.000000
1    1.333333
2    1.428571
3    1.666667
4    2.000000
Length: 1725, dtype: float64

In [191]:
(full_df
 .assign(ratio_info_items_extracted=lambda df: df['total_info_items_extracted'] / df['total_info_item_count'] )
 # .loc[lambda df: df['ratio_info_items_extracted'] < 1]
 .groupby(['trial_name', 'game_level'])
 ['ratio_info_items_extracted']
 .mean()
 .unstack()
)

game_level,game_level_advanced,game_level_basic,game_level_intermediate
trial_name,,,
interviewer_gpt-4o-mini_vs_source_gpt-4o,1.301663,1.437212,1.386291
interviewer_gpt-4o_vs_source_gpt-4o,1.239854,1.438695,1.394701
interviewer_meta-llama_Meta-Llama-3.1-70B-Instruct_vs_source_gpt-4o,1.288307,1.395966,1.287602
interviewer_meta-llama_meta-llama-3.1-8b-instruct_vs_source_gpt-4o,0.345714,1.426886,1.284531


In [164]:
running_convo_df = pd.DataFrame(full_df['running_conversations'].to_list())
running_convo_df[['trial_name', 'game_level']] = full_df[['trial_name', 'game_level']].reset_index(drop=True)

In [165]:
info_num_columns = list(filter(lambda x: 'info_item_num' in x , running_convo_df.columns))
info_desc_columns = list(filter(lambda x: 'info_items_desc' in x , running_convo_df.columns))
persuasion_columns = list(filter(lambda x: 'persuasion' in x , running_convo_df.columns))

In [175]:
running_convo_df[info_num_columns].apply(lambda s: s.str.len()).sum(axis=1).value_counts()

0    1473
1     115
2      58
3      32
4      18
5      14
6       7
7       5
9       2
8       1
Name: count, dtype: int64

In [177]:
running_convo_df['total_info_items'] = running_convo_df[info_items_columns].apply(lambda s: s.str.len()).fillna(0).sum(axis=1)

In [170]:
(running_convo_df
 .groupby(['trial_name', 'game_level'])
 ['total_info_items']
 .mean()
 .unstack()
)

game_level,game_level_advanced,game_level_basic,game_level_intermediate
trial_name,,,
interviewer_gpt-4o-mini_vs_source_gpt-4o,0.500000,0.331250,0.346667
interviewer_gpt-4o_vs_source_gpt-4o,0.505263,0.331250,0.353333
interviewer_meta-llama_Meta-Llama-3.1-70B-Instruct_vs_source_gpt-4o,0.476190,0.291429,0.312500
interviewer_meta-llama_meta-llama-3.1-8b-instruct_vs_source_gpt-4o,2.000000,0.212000,0.227907


In [167]:
running_convo_df[info_items_columns]

,info_item_num_1,info_item_num_2,info_item_num_3,info_item_num_4,info_item_num_5,info_item_num_6,info_item_num_7
0,[4],[],[5],[2],[],[],[]
1,[],[],[],[],[],[],[]
2,[5],[4],[],"[2, 3]",[],[],[]
3,[3],[2],[7],[],[],[],[]
4,[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...
1720,[],[],[],[9],[],[],[]
1721,[],[],[],[],[],[],[]
1722,[],[],[],[],[],[],[]
1723,[],[],[],[],[],[],[]


# Look at sample DF

In [153]:
df = pd.read_json('../test/conducted_interviews_batch_0_2.jsonl', lines=True)

In [156]:
pd.DataFrame(df['running_conversations'].tolist())

,question_0,persuasion_0,info_items_0,answer_0,question_1,persuasion_1,info_items_1,answer_1,question_2,persuasion_2,info_items_2,answer_2,question_3,persuasion_3,info_items_3,answer_3,question_4,answer_4
0,"Today, we have the opportunity to speak with a...",None,None,Thank you for the introduction. It's always in...,“What has been the predominant reaction within...,1,[No relevant information for this question. Fe...,"Well, you know, reactions can vary widely. Som...","“I understand the complexity, but could you sh...",1,[Information Item #4: The Russian government i...,"Well, there hasn't been a major outcry from th...",“I appreciate your insights on the broader con...,2,[All relevant information has been given to th...,"You know, there may be some impact on public p...",Thank you for providing your valuable insights...,Thank you for having me. It’s always great to ...
1,"Today, we're diving into the final year of the...",None,None,Thank you for the introduction. I'm glad to be...,"“In your opinion, what do you think will be th...",1,[Information Item #4: President Bush will like...,"In my view, the most significant aspect of Pre...",“Looking at the upcoming foreign policy decisi...,1,[Information Item #3: President Bush is planni...,Certainly. I anticipate that the current admin...,"“In your opinion, what may be some potential s...",1,[No relevant information for this question. Fe...,While I don’t have specific information on sur...,Thank you for sharing your insights and analys...,Thank you for the opportunity to discuss these...
